<a href="https://colab.research.google.com/github/regza22/nikitakotov/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_3_%D1%81%D0%B5%D0%BC%D0%B5%D1%81%D1%82%D1%80_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1

In [ ]:
from fastapi import FastAPI, Request, Form, HTTPException
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse, RedirectResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

templates = Jinja2Templates(directory='templates')
app.mount('/static', StaticFiles(directory='static'), name='static')

class Note(BaseModel):
    id: int
    title: str
    content: str

notes = []
note_counter = 0  # Счетчик для ID заметок

@app.get('/', response_class=HTMLResponse)
def get_index(request: Request):
    return templates.TemplateResponse(
        'index.html',
        {'request': request, 'notes': notes}
    )

@app.get('/notes', response_model=List[Note])
def get_all_notes():
    return notes

@app.get('/notes/{note_id}', response_model=Note)
def get_note_by_id(note_id: int):
    for note in notes:
        if note.id == note_id:
            return note
    raise HTTPException(status_code=404, detail="Note not found")

@app.get('/notes/stats')
def get_notes_stats():
    return {"count": len(notes)}

@app.post('/notes', response_model=Note, status_code=201)
def create_note(note_title: str = Form(...), note_content: str = Form(...)):
    global note_counter
    note_counter += 1
    new_note = Note(id=note_counter, title=note_title, content=note_content)
    notes.append(new_note)
    return RedirectResponse(url="/", status_code=303)

@app.put('/notes/{note_id}', response_model=Note)
def update_note(note_id: int, updated_note: Note):
    for index, note in enumerate(notes):
        if note.id == note_id:
            notes[index] = updated_note
            return updated_note
    raise HTTPException(status_code=404, detail="Note not found")

@app.delete('/notes/{note_id}', status_code=204)
def delete_note(note_id: int):
    for index, note in enumerate(notes):
        if note.id == note_id:
            notes.pop(index)
            return
    raise HTTPException(status_code=404, detail="Note not found")

@app.delete('/notes', status_code=204)
def delete_all_notes():
    notes.clear()